<a href="https://colab.research.google.com/github/MatheusLCouto/TAIA/blob/main/Extrator_de_Aspectos_Ride_hailing(Pipeline_experimental).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dividir CSV em Partes proporcionais

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import unicodedata

In [ ]:
def filter_ubereats_complaints(input_path: str, output_path: str, text_column: str = "report_text"):

    keywords = [
        "entregador", "motoboy", "comida", "almoço", "almoco",
        "pedido", "encomenda", "falha na entrega", "eats", "Uber-eats", "Eats"
    ]

    pattern = r"\b(" + "|".join(keywords) + r")\b"

    def normalize(text: str) -> str:
        text = str(text).lower()
        return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

    def is_ubereats_related(text: str) -> bool:
        return bool(re.search(pattern, normalize(text)))


    df = pd.read_csv(input_path)
    original_count = len(df)

    filtered_df = df[~df[text_column].apply(is_ubereats_related)].copy()

    filtered_df.to_csv(output_path, index=False)

    removed = original_count - len(filtered_df)

    print(f"Removido: {removed} reclamacoes")
    print(f"Salvo em: {output_path}")

In [ ]:
filter_ubereats_complaints(
    input_path="/content/drive/MyDrive/ride_hailing/Data/Uber_complaint.csv",
    output_path="/content/drive/MyDrive/ride_hailing/Data/Uber_complaint_filtered.csv",
    text_column="report_text"
)

Removido: 11986 reclamacoes
Salvo em: /content/drive/MyDrive/ride_hailing/Data/Uber_complaint_filtered.csv


In [ ]:
App = pd.read_csv('/content/drive/MyDrive/ride_hailing/Data/99App_complaint.csv')
Uber = pd.read_csv('/content/drive/MyDrive/ride_hailing/Data/Uber_complaint_filtered.csv')

In [ ]:
df_ride= pd.concat([App, Uber])

In [ ]:
df_ride.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39412 entries, 0 to 30636
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name_company            39412 non-null  object
 1   report_status           39412 non-null  object
 2   report_text             39412 non-null  object
 3   report_date             39412 non-null  object
 4   reponse_company         39412 non-null  object
 5   reponse_company_date    39412 non-null  object
 6   service_note            39412 non-null  object
 7   consumer_consideration  39411 non-null  object
dtypes: object(8)
memory usage: 2.7+ MB


In [ ]:
df_ride.to_csv('df_ride.csv', index=False)

In [ ]:
df_ride = pd.read_csv('df_ride.csv')

In [ ]:
# import pandas as pd

# # Função para dividir o dataset proporcionalmente
# def dividir_dataset_proporcional(dataset, tamanho_parte, coluna_empresa):
#     partes = []  # Lista para armazenar as partes
#     dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)  # Embaralhar o dataset

#     # Iterar até que o dataset esteja vazio
#     while not dataset.empty:
#         parte_atual = pd.DataFrame()  # Inicializar a parte atual
#         empresas = dataset[coluna_empresa].unique()  # Empresas únicas no dataset restante

#         # Selecionar amostras proporcionalmente para cada empresa
#         for empresa in empresas:
#             subset_empresa = dataset[dataset[coluna_empresa] == empresa]
#             proporcao = len(subset_empresa) / len(dataset)
#             tamanho_amostra = int(round(proporcao * tamanho_parte))

#             # Selecionar amostras
#             amostra = subset_empresa.head(tamanho_amostra)
#             parte_atual = pd.concat([parte_atual, amostra])

#             # Remover amostras do dataset original
#             dataset = dataset.drop(amostra.index)

#         # Adicionar a parte atual à lista de partes
#         partes.append(parte_atual)

#     return partes

# # Carregar o dataset (substitua 'seu_arquivo.csv' pelo arquivo real se necessário)
# complete = pd.read_csv("df_ride.csv")  # Certifique-se de que o arquivo contém os campos especificados

# # Dividindo o dataset em partes de 3.000 linhas
# partes = dividir_dataset_proporcional(complete, tamanho_parte=6000, coluna_empresa="name_company")

# # Verificando os resultados
# for i, parte in enumerate(partes):
#     nome_arquivo = f"subset_parte_{i + 1}.csv"
#     parte.to_csv(nome_arquivo, index=False)
#     print(f"Salvo: {nome_arquivo}")

Carregar Ollama

In [ ]:
import pandas as pd
import os

# Caminho da pasta no Google Drive
pasta_destino = "/content/drive/MyDrive/ride_hailing/Subsets"

# Criar a pasta caso não exista
os.makedirs(pasta_destino, exist_ok=True)

# Função para dividir o dataset proporcionalmente
def dividir_dataset_proporcional(dataset, tamanho_parte, coluna_empresa):
    partes = []
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    while not dataset.empty:
        parte_atual = pd.DataFrame()
        empresas = dataset[coluna_empresa].unique()

        for empresa in empresas:
            subset_empresa = dataset[dataset[coluna_empresa] == empresa]
            proporcao = len(subset_empresa) / len(dataset)
            tamanho_amostra = int(round(proporcao * tamanho_parte))

            amostra = subset_empresa.head(tamanho_amostra)
            parte_atual = pd.concat([parte_atual, amostra])
            dataset = dataset.drop(amostra.index)

        partes.append(parte_atual)

    return partes

# Carregar o dataset
complete = pd.read_csv("df_ride.csv")

# Dividindo o dataset
partes = dividir_dataset_proporcional(complete, tamanho_parte=6000, coluna_empresa="name_company")

# Salvando os arquivos no Google Drive
for i, parte in enumerate(partes):
    nome_arquivo = f"subset_parte_{i + 1}.csv"
    caminho_completo = os.path.join(pasta_destino, nome_arquivo)
    parte.to_csv(caminho_completo, index=False)
    print(f"Salvo: {caminho_completo}")


Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_1.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_2.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_3.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_4.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_5.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_6.csv
Salvo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_7.csv


In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  85721      0 --:--:-- --:--:-- --:--:-- 86240
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &> ollama.serve.log &

In [ ]:
!cat ollama.serve.log

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMozRhQtVGCIPDCr2JKuM+tmDp3gnqhIHZX76wTrwN+H

time=2025-06-21T13:53:59.348Z level=INFO source=routes.go:1235 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.

In [ ]:
# mudar o modelo como no exemplo, alterando apos o run
# !nohup ollama run deepseek-coder:latest > deepseek.log &
!nohup ollama run llama3.1 > llama3.1.log &

nohup: redirecting stderr to stdout


In [ ]:
# !ollama run llama3.1:8b-text-fp16

In [ ]:
# se mudar o modelo, precisa mudar o cat
# !cat deepseek.log
!cat llama3.1.log
# !cat llama3.1:8b-text-fp16

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ollama create ride_aspects -f /content/drive/MyDrive/ride_hailing/ModelFile

In [ ]:
!ollama list

NAME                   ID              SIZE      MODIFIED               
ride_aspects:latest    c7419aedccce    4.9 GB    Less than a second ago    
llama3.1:latest        46e0c10c039e    4.9 GB    Less than a second ago    


In [ ]:
df_ride['report_text'].loc[11]

'Solicitei a corrida, o motoqueiro desviou da rota prevista. Um terço do caminho percorrido teve problemas mecânico. O motoqueiro encerrou a viagem como se tivesse finalizada corretamente.'

In [ ]:
#certo
import json
import openai

openai.api_key = "dummy"
openai.api_base = "http://127.0.0.1:11434/v1"

def llm_laca_task(task, text, max_tokens=500):
    # Combine o texto e a tarefa no formato esperado
    prompt = f"{text}\n===\n{task}"

    try:
        # Faz a chamada ao modelo
        response = openai.ChatCompletion.create(
            model="ride_aspects",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        # Extrai o conteúdo da resposta
        full_text = response['choices'][0]['message']['content']
        return full_text

    except Exception as e:
        print("Error during API call:", e)
        return None


In [ ]:
#  texto ='''O VOO AZUL AD 2546 (CONFINS/UNA) TINHA HORÁRIO PREVISTO PARA DECOLAGEM AS 13:15HS POREM SÓ DECOLOU AS 16:04HS COM 02:49HS DE ATRASO. COM ISTO, A CHEGADA NO DESTINO OCORREU SOMENTE AS 17:31HS, PERDENDO ASSIM PRATICAMENTE A DIARIA'''
#  # '''relato que em 26/02/2023 adquiri 02 (dois) bilhetes aéreos - para mim e minha esposa - no site da própria LATAM nos seguintes moldes:Código de reserva: UTNBVI Número de ordem: LA9571039EYTFPassageiros: RODRIGO PIOVEZAN (Passaporte - FZ641301) e NATALIA COLLI (Passaporte   FZ341308)Trecho: CGB(Cuiabá/MT)   LAS (Las Vegas/EUA), com conexão em GRU (Guarulhos/SP) e ATL (Atlanta/EUA)Data do voo: 29/08/2023VALOR PAGO: 178.032 pontos LATAMPASS + R$ 705,08.Destaco que recebi os bilhetes eletrônicos normalmente e, na sequência, realizei nossa reserva e pagamento de hotel em LAS VEGAS.Contudo, consultando meu app LATAM, constatei que HOUVE UMA ALTERAÇÃO UNILATERAL NO MEU DESTINO FINAL SEM O MEU CONSENTIMENTO, pois, adquiri bilhetes com destino final LAS VEGAS e, agora, aparece no sistema como destino final ATLANTA, o que não foi o contratado e pago.Ocorre que não concordo com a alteração, até porque o destino final ATLANTA NÃO FOI O CONTRATADO E PAGO, MAS SIM, LAS VEGAS.Diante dessa descoberta, entrei em contato com a LATAM via FALE CONOSCO e formalizei minha reclamação em 05/06/2023 através do REGISTRO DE CASO #51357364, onde SOLICITEI PROVIDÊNCIAS URGENTES acerca da correção de meu destino final efetivamente comprado, isto é, para que volte a constar nos bilhetes adquiridos o destino final LAS VEGAS.No entanto, em data de 08/06/2023, recebi por e-mail a absurda resposta da LATAM informando o quanto segue:  Peço desculpa pelo transtorno causado. Referente ao destino final em Las Vegas o voo foi cancelado pela empresa Ibéria. Peço que entre em contato com a Empresa Ibéria para estar verificando o procedimento, pois como é companhia parceira, não temos acesso ao voo cancelado. Sendo assim daremos este caso como encerrado. .Ocorre que TODO O TRECHO (CUIABÁ   LAS VEGAS) foi adquirido junto à LATAM, não tendo havido qualquer contato deste consumidor com a empresa aérea IBÉRIA, a qual nem sequer constava em meu bilhete como operadora do voo, mas sim, a empresa DELTA, a qual faria o trecho ATL-LAS no voo LA8494.Desse modo, através dessa plataforma do Governo, SOLICITO PROVIDÊNCIAS URGENTES por parte da LATAM, a fim de que a mesma retorne o trecho final LAS VEGAS para a minha reserva, ainda que sejam necessária adequação de horários, uma vez ser obrigação da companhia aérea honrar com o fornecimento dos bilhetes nos moldes em que fora por mim contratado e pago.Reitero que o destino final por mim adquirido sempre foi LAS VEGAS, e não ATLANTA, SENDO INADMISSÍVEL QUE A LATAM ME DEIXE NO MEIO DO CAMINHO'''

# llm_laca_task("""
# [Persona]:
# Você é especialista em extração de aspectos e análise de reclamações no domínio de companhias aéreas. Baseie suas respostas estritamente nos textos fornecidos e no prompt fornecido, sem adicionar nenhuma informação extra.
# [Tarefa]:
# Extraia apenas as *palavras*, da maneira que estão escritas, que representam aspectos no texto de reclamação. Aspectos são características, atributos ou componentes específicos de um produto, serviço ou experiência mencionados no texto.
# [Objetivo]:
# Exibir somente as palavras correspondentes aos aspectos encontrados no texto, sem explicações ou informações adicionais.
# [Modelo]:
# A resposta deve ser uma lista em português, contendo somente as palavras identificadas como aspectos.
# [Panorama]:
# O texto da reclamação será fornecido pelo usuário, e o modelo deve se concentrar em aspectos como 'preço', 'qualidade', 'atendimento',' produto', entre outros termos relacionados.
# [Resposta]:
# Com base no texto mostre as palavras aspectos extraídos no seguinte formato: [aspecto1 , aspecto2 , aspecto3 ]
# """, texto)


In [ ]:
 texto = '''Solicitei a corrida, o motoqueiro desviou da rota prevista. Um terço do caminho percorrido teve problemas mecânico. O motoqueiro encerrou a viagem como se tivesse finalizada corretamente'''

llm_laca_task("""
[Persona]
  Você é um especialista em processamento de linguagem natural focado em análise textual e extração precisa de aspectos em textos de reclamação.
[Tarefa]
  Extraia **apenas as palavras** que representam **aspectos** mencionados no texto fornecido.
[Contexto]
  Aspectos são características, atributos ou componentes específicos relacionados a um produto, serviço ou experiência. Esses aspectos podem incluir elementos concretos (como "durabilidade") ou subjetivos (como "satisfação").
[Instruções]
  1. Analise o texto fornecido e identifique os aspectos presentes.
  2. Liste **somente a palavra** que representam os aspectos, sem adicionar explicações, introduções ou classificações.
[Formato de Resposta]
  - A resposta deve ser uma **lista simples** em português.
  - **Cada aspecto deve aparecer em uma nova linha.**
  - **Sem contexto, explicações ou textos adicionais.**
[Exemplo de Resposta]
  - Preço
  - Atendimento
  - Qualidade
  - Funcionalidade
  - Design
[Panorama do Texto]
  O texto analisado será uma reclamação ou avaliação de cliente, onde aspectos como **custo**, **usabilidade**, **serviço** e **durabilidade** são comuns. Identifique-os com precisão e mantenha o foco na tarefa.
""", texto)


'- Motoqueiro\n- Rota\n- Problema mecânico\n- Viagem\n- Corrida'

In [ ]:
texto = '''relato que em 26/02/2023 adquiri 02 (dois) bilhetes aéreos - para mim e minha esposa - no site da própria LATAM nos seguintes moldes:Código de reserva: UTNBVI Número de ordem: LA9571039EYTFPassageiros: RODRIGO PIOVEZAN (Passaporte - FZ641301) e NATALIA COLLI (Passaporte   FZ341308)Trecho: CGB(Cuiabá/MT)   LAS (Las Vegas/EUA), com conexão em GRU (Guarulhos/SP) e ATL (Atlanta/EUA)Data do voo: 29/08/2023VALOR PAGO: 178.032 pontos LATAMPASS + R$ 705,08.Destaco que recebi os bilhetes eletrônicos normalmente e, na sequência, realizei nossa reserva e pagamento de hotel em LAS VEGAS.Contudo, consultando meu app LATAM, constatei que HOUVE UMA ALTERAÇÃO UNILATERAL NO MEU DESTINO FINAL SEM O MEU CONSENTIMENTO, pois, adquiri bilhetes com destino final LAS VEGAS e, agora, aparece no sistema como destino final ATLANTA, o que não foi o contratado e pago.Ocorre que não concordo com a alteração, até porque o destino final ATLANTA NÃO FOI O CONTRATADO E PAGO, MAS SIM, LAS VEGAS.Diante dessa descoberta, entrei em contato com a LATAM via FALE CONOSCO e formalizei minha reclamação em 05/06/2023 através do REGISTRO DE CASO #51357364, onde SOLICITEI PROVIDÊNCIAS URGENTES acerca da correção de meu destino final efetivamente comprado, isto é, para que volte a constar nos bilhetes adquiridos o destino final LAS VEGAS.No entanto, em data de 08/06/2023, recebi por e-mail a absurda resposta da LATAM informando o quanto segue:  Peço desculpa pelo transtorno causado. Referente ao destino final em Las Vegas o voo foi cancelado pela empresa Ibéria. Peço que entre em contato com a Empresa Ibéria para estar verificando o procedimento, pois como é companhia parceira, não temos acesso ao voo cancelado. Sendo assim daremos este caso como encerrado. .Ocorre que TODO O TRECHO (CUIABÁ   LAS VEGAS) foi adquirido junto à LATAM, não tendo havido qualquer contato deste consumidor com a empresa aérea IBÉRIA, a qual nem sequer constava em meu bilhete como operadora do voo, mas sim, a empresa DELTA, a qual faria o trecho ATL-LAS no voo LA8494.Desse modo, através dessa plataforma do Governo, SOLICITO PROVIDÊNCIAS URGENTES por parte da LATAM, a fim de que a mesma retorne o trecho final LAS VEGAS para a minha reserva, ainda que sejam necessária adequação de horários, uma vez ser obrigação da companhia aérea honrar com o fornecimento dos bilhetes nos moldes em que fora por mim contratado e pago.Reitero que o destino final por mim adquirido sempre foi LAS VEGAS, e não ATLANTA, SENDO INADMISSÍVEL QUE A LATAM ME DEIXE NO MEIO DO CAMINHO'''
llm_laca_task("task = '''Extract only the **WORDS** that represent aspects in the complaint text. Aspects are specific characteristics, attributes, or components of a product, service, or experience mentioned in the text. **Display only the words, without explanations or additional information, In Portuguese**.'''", texto)

'• Bilhetes\n• Passaporte\n• Trecho\n• Voo\n• Conexão\n• Data\n• Valor\n• Pagamento\n• Destino final\n• App\n• Sistema\n• Alteração unilateral\n• Consentimento\n• Destino final LAS VEGAS\n• Destino final ATLANTA\n• Hotel\n• Reclamação\n• Providências urgentes\n• Correção\n• E-mail\n• Transtorno\n• Companhia parceira\n• Acesso\n• Voo cancelado\n• Trecho CUIABÁ LAS VEGAS\n• Operadora do voo\n• Adequação de horários\n• Fornecimento dos bilhetes'

In [ ]:
#Código original
# task ='''Extract only the **WORDS** that represent aspects in the complaint text. Aspects are specific characteristics, attributes, or components of a product, service, or experience mentioned in the text. **Display only the words, without explanations or additional information,In Portuguese**.'''
# #Extract only the words that represent aspects in the complaint text. Display only the words, without explanations or additional information.

# dic_counter = {}

# L_resp = []

# for index,row in df_sample.iterrows():
#   text = '\n\n###\n@Document\n\n'+row['report_text_decode']
#   try_run = 0
#   while True:
#     try:
#       L = llm_laca_task(task,text)
#       if isinstance(L, str):
#         # print(text)
#         topic = L[0:].strip()
#         # topic = L[0:2].strip()
#         print('Topic:',topic)
#         L_resp.append(topic)
#         print("=========================")
#         break
#     except:
#       1
#     try_run += 1
#     if try_run >= 5:
#       L_resp.append('NONE')
#       break

# print(L_resp)

Extrair Aspectos por CSV salvando a cada execução

In [ ]:
#teste de prompt
import pandas as pd
import glob
from tqdm import tqdm

# Função para processar os arquivos CSV
def processar_arquivos(pasta_gdrive, pasta__saida, task):
    # Localiza todos os arquivos CSV na pasta especificada
    arquivos = glob.glob(f"{pasta_gdrive}/*.csv")

    print(f"Encontrados {len(arquivos)} arquivos para processamento na pasta: {pasta_gdrive}")

    for arquivo in arquivos:
        print(f"\nProcessando arquivo: {arquivo}")

        # Carregar o dataset
        df_sample = pd.read_csv(arquivo)

        # Lista para armazenar os resultados processados
        L_resp = []

        # Iterar sobre cada linha do DataFrame com barra de progresso
        for index, row in tqdm(df_sample.iterrows(), total=df_sample.shape[0], desc=f"Processando {arquivo.split('/')[-1]}"):
            text = f"\n\n###\n@Document\n\n{row['report_text']}"
            try_run = 0
            while True:
                try:
                    # Executando a tarefa de extração de palavras
                    L = llm_laca_task(task, text)  # Substitua pelo método adequado para a execução no ambiente
                    if isinstance(L, str):
                        topic = L.strip()  # Remover espaços extras
                     #   print(f"\n--- Resultado extraído para a linha {index} ---\n{topic}\n")
                        L_resp.append(topic)
                        break

                except Exception as e:
                    print(f"Erro ao processar o texto: {e}")
                try_run += 1
                if try_run >= 5:
                    L_resp.append('NONE')
                    break

        # Adicionar a nova coluna com os resultados ao DataFrame
        df_sample['document_topic'] = L_resp

        # Gerar o nome do arquivo processado
        nome_arquivo = f"{pasta__saida}/processed_{arquivo.split('/')[-1]}"

        # Salvar o arquivo processado em CSV
        df_sample.to_csv(nome_arquivo, index=False)
        print(f"Arquivo processado e salvo como: {nome_arquivo}")

# Montar o Google Drive no Colab
from google.colab import drive
drive.mount('/content/drive')

# Caminho para a pasta no Google Drive
pasta_gdrive = "/content/drive/MyDrive/ride_hailing/Subsets"
pasta__saida = "/content/drive/MyDrive/ride_hailing/Results"

# Task definida
task =  """
[Persona]
  Você é um especialista em processamento de linguagem natural focado em análise textual e extração precisa de *Issues* em textos de reclamação.
[Tarefa]
  Extraia **apenas as palavras** que representam **Issues** mencionados no texto fornecido.
[Contexto]
  *Issues* são características, atributos ou componentes específicos relacionados a um produto, serviço ou experiência. Esses *Issues* podem incluir elementos concretos como:
  - Funcionamento do app (ex: falhas técnicas, travamentos)
  - Motoristas (ex: comportamento, direção perigosa)
  - Preço (ex: cobranças indevidas, tarifas altas)
  - Tempo de espera, atrasos ou rotas ruins
  - Segurança, atendimento ou qualquer outro ponto de insatisfação.
[Instruções]
  1. Analise o texto fornecido e identifique os *Issues* presentes.
  2. Liste **somente a palavra** que representam os *Issues*, sem adicionar explicações, introduções ou classificações.
[Formato de Resposta]
- A resposta deve ser **uma lista simples**, apenas com as palavras que representam os *Issues*, **sem qualquer texto antes ou depois**.
- **Cada *Issue* deve aparecer em uma nova linha, sozinho.**
- **Não inclua introduções, títulos, resumos ou frases explicativas. Apenas a lista crua.**
[Exemplo de Resposta]
  - Cobrança indevida
  - Atraso
  - Aplicativo travando
  - Motorista mal-educado
  - Insegurança
[Panorama do Texto]
  O texto analisado será uma reclamação ou avaliação de cliente, onde *Issues* como **custo**, **usabilidade**, **serviço** e **durabilidade** são comuns. Identifique-os com precisão e mantenha o foco na tarefa.
"""
# Processar os arquivos na pasta especificada


processar_arquivos(pasta_gdrive, pasta__saida, task)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Encontrados 7 arquivos para processamento na pasta: /content/drive/MyDrive/ride_hailing/Subsets

Processando arquivo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_1.csv


Processando subset_parte_1.csv: 100%|██████████| 6164/6164 [3:57:36<00:00,  2.31s/it]


Arquivo processado e salvo como: /content/drive/MyDrive/ride_hailing/Results/processed_subset_parte_1.csv

Processando arquivo: /content/drive/MyDrive/ride_hailing/Subsets/subset_parte_2.csv


Processando subset_parte_2.csv:   1%|          | 61/6195 [02:24<3:32:27,  2.08s/it]

Plotar Gráfico com os 20 aspectos mais encontrados (tratados)

In [ ]:
####CCEERRRRRRRTTOOOOOOOOOO Antigo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import nltk
import re
import unicodedata

df = pd.read_csv('/content/drive/MyDrive/ASPECTS/ride_hailing/Subsets/processed_subset_parte_1.csv')
# Baixar os recursos necessários para lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

# Inicializar o lemmatizer
lemmatizer = WordNetLemmatizer()

# Função para normalizar palavras
def normalize_text(text):
    # Remover acentuação
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Transformar para minúsculas
    text = text.lower()
    # Remover caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Remover espaços extras
    text = text.strip()
    # Aplicar lemmatization
    text = lemmatizer.lemmatize(text)
    return text

# Carregar o dataset
#df = pd.read_csv('processed_subset_parte_1.csv')

# Dividir a coluna `document_topic` por linha ("/n") e criar colunas separadas
topics_split = df['document_topic'].str.split('\n', expand=True)

# Anexar as novas colunas ao DataFrame original
df = pd.concat([df, topics_split], axis=1)

# Identificar as colunas criadas a partir de `document_topic`
topic_columns = topics_split.columns

# Empilhar todas as colunas de tópicos em uma única série
topics_flat = df[topic_columns].stack()

# Remover valores nulos
topics_flat = topics_flat.dropna()

# Aplicar a normalização em cada palavra
topics_flat = topics_flat.apply(normalize_text)

# Substituir palavras mal grafadas ou variações manuais
topics_flat = topics_flat.replace({
    "passagen": "passagem",
    "passagens": "passagem",
    "voos": "voo",
    "bilhetes": "bilhete",
    # Adicione outros mapeamentos se necessário
})

# Contar as ocorrências de palavras únicas
word_counts = topics_flat.value_counts()

# Exibir o número total de palavras únicas extraídas
total_words = word_counts.shape[0]
print(f"Número total de palavras únicas extraídas: {total_words}")

# Transformar a saída em um DataFrame
df_table = word_counts.reset_index()
df_table.columns = ["Palavra", "Contagem"]

# Exibir os 20 primeiros registros para validação
print(df_table.head(100))

# Criar um gráfico de barras para as top 20 palavras
plt.figure(figsize=(20, 10))
ax = sns.barplot(x="Contagem", y="Palavra", data=df_table.head(20), palette="viridis")
plt.title("Top 20 Aspectos mais frequentes Primeira versão", fontsize= 15)
plt.xlabel("Ocorrência", fontsize=15)
plt.ylabel("Aspecto", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=18)
plt.tight_layout()

# Adicionar os valores ao lado de cada barra
for i, value in enumerate(df_table.head(20)["Contagem"]):
    ax.text(value + 1, i, str(value), color="black", va="center", fontsize=15)


# Salvar o gráfico
plt.savefig("grafico_barras_top_20_palavras_normalizadas.png")
plt.show()

# Salvar a tabela com todas as palavras
df_table.to_csv("contagem_palavras_completa_norm_30k.csv", index=False)

print("Tabela de contagem de palavras salva com sucesso!")


In [ ]:
####Nova versão normalização
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import nltk
import re
import unicodedata

# Baixar os recursos necessários para lemmatization e stopwords
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Inicializar o lemmatizer
lemmatizer = WordNetLemmatizer()

# Função para normalizar palavras
def normalize_text(text):
    # Remover acentuação
    text_normalized = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Transformar para minúsculas
    text_normalized = text_normalized.lower()
    # Remover caracteres especiais
    text_normalized = re.sub(r'[^\w\s]', '', text_normalized)
    # Remover espaços extras
    text_normalized = text_normalized.strip()

    # Substituir variações de "taxa" diretamente
    if re.search(r'\btax[a-z]*\b', text_normalized):  # Qualquer palavra que comece com 'tax'
        if text_normalized != "taxon":
            text_normalized = 'taxa'

    # Aplicar lemmatization
    text_normalized = lemmatizer.lemmatize(text_normalized)

    # Garantir que "taxa" não seja transformado em "taxon"
    if text_normalized == "taxon":
        text_normalized = "taxa"

    return text_normalized

# Carregar o dataset
df = pd.read_csv('/content/drive/My Drive/ASPECTS/Teste_prompt/processed_Cópia de subset_parte_1.csv')

# Dividir a coluna `document_topic` por linha ("/n") e criar colunas separadas
topics_split = df['document_topic'].str.split('\n', expand=True)

# Anexar as novas colunas ao DataFrame original
df = pd.concat([df, topics_split], axis=1)

# Identificar as colunas criadas a partir de `document_topic`
topic_columns = topics_split.columns

# Empilhar todas as colunas de tópicos em uma única série
topics_flat = df[topic_columns].stack()

# Remover valores nulos
topics_flat = topics_flat.dropna()

# Aplicar a normalização em cada palavra
topics_flat_normalized = topics_flat.apply(normalize_text)

# Verificar as palavras relacionadas a 'taxon' e 'taxa' após normalização
taxon_occurrences = topics_flat_normalized[topics_flat_normalized.str.contains(r'\btaxon\b', case=False)]

print(f"Ocorrências de 'taxon' após normalização: {taxon_occurrences.shape[0]}")

# Contar as ocorrências de palavras únicas
word_counts = topics_flat_normalized.value_counts()

# Exibir o número total de palavras únicas extraídas
total_words = word_counts.shape[0]
print(f"Número total de palavras únicas extraídas: {total_words}")

# Transformar a saída em um DataFrame
df_table = word_counts.reset_index()
df_table.columns = ["Palavra", "Contagem"]

# Exibir as 50 primeiras palavras mais frequentes para validação
# print(df_table.head(50))
print(df_table.head(20))
# Criar um gráfico de barras para as 50 palavras mais frequentes
plt.figure(figsize=(20, 10))
ax = sns.barplot(x="Contagem", y="Palavra", data=df_table.head(20), palette="viridis")
plt.title("Top 20 Aspectos mais frequentes", fontsize= 15)
plt.xlabel("Ocorrência", fontsize=15)
plt.ylabel("Aspecto", fontsize=15)
plt.xticks(fontsize=12)#12
plt.yticks(fontsize=18)#18
plt.tight_layout()

# Adicionar os valores ao lado de cada barra
for i, value in enumerate(df_table.head(20)["Contagem"]):
    ax.text(value + 1, i, str(value), color="black", va="center", fontsize=15)

# Salvar o gráfico
plt.savefig("grafico_barras_top_20_palavras_normalizadas.png")
plt.show()

# Salvar a tabela com todas as palavras
df_table.to_csv("contagem_palavras_completa_norm_20.csv", index=False)

print("Tabela de contagem de palavras salva com sucesso!")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import nltk
import re
import unicodedata

# Baixar os recursos necessários para lemmatization e stopwords
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Inicializar o lemmatizer
lemmatizer = WordNetLemmatizer()

# Função para normalizar palavras
def normalize_text(text):
    # Remover acentuação
    text_normalized = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Transformar para minúsculas
    text_normalized = text_normalized.lower()
    # Remover caracteres especiais
    text_normalized = re.sub(r'[^\w\s]', '', text_normalized)
    # Remover espaços extras
    text_normalized = text_normalized.strip()

    # Substituir variações de "taxa" diretamente
    if re.search(r'\btax[a-z]*\b', text_normalized):  # Qualquer palavra que comece com 'tax'
        if text_normalized != "taxon":
            text_normalized = 'taxa'

    # Aplicar lemmatization
    text_normalized = lemmatizer.lemmatize(text_normalized)

    # Garantir que "taxa" não seja transformado em "taxon"
    if text_normalized == "taxon":
        text_normalized = "taxa"

    return text_normalized

# Carregar o dataset
df = pd.read_csv('/content/drive/My Drive/ASPECTS/Teste_prompt/processed_Cópia de subset_parte_1.csv')

# Dividir a coluna `document_topic` por linha ("/n") e criar colunas separadas
topics_split = df['document_topic'].str.split('\n', expand=True)

# Anexar as novas colunas ao DataFrame original
df = pd.concat([df, topics_split], axis=1)

# Identificar as colunas criadas a partir de `document_topic`
topic_columns = topics_split.columns

# Contar palavras antes da normalização
# Empilhar todas as colunas de tópicos em uma única série
topics_flat_before = df[topic_columns].stack()

# Remover valores nulos
topics_flat_before = topics_flat_before.dropna()

# Contar o número total de palavras antes da normalização
word_counts_before = topics_flat_before.value_counts()
total_words_before = word_counts_before.shape[0]
print(f"Número total de palavras únicas antes da normalização: {total_words_before}")

# Aplicar a normalização em cada palavra
topics_flat_normalized = topics_flat_before.apply(normalize_text)

# Contar o número total de palavras após a normalização
word_counts_after = topics_flat_normalized.value_counts()
total_words_after = word_counts_after.shape[0]
print(f"Número total de palavras únicas após a normalização: {total_words_after}")

# Verificar as palavras relacionadas a 'taxon' e 'taxa' após normalização
taxon_occurrences = topics_flat_normalized[topics_flat_normalized.str.contains(r'\btaxon\b', case=False)]
print(f"Ocorrências de 'taxon' após normalização: {taxon_occurrences.shape[0]}")

# Transformar a saída em um DataFrame para palavras normalizadas
df_table_normalized = word_counts_after.reset_index()
df_table_normalized.columns = ["Palavra", "Contagem"]

# Criar um DataFrame para comparar palavras antes e depois
comparison_table = pd.DataFrame({
    "Palavra Antes": word_counts_before.index,
    "Contagem Antes": word_counts_before.values
}).merge(
    pd.DataFrame({
        "Palavra Depois": word_counts_after.index,
        "Contagem Depois": word_counts_after.values
    }),
    left_on="Palavra Antes",
    right_on="Palavra Depois",
    how="outer"
).fillna(0)

# Salvar a tabela de comparação
comparison_table.to_csv("comparacao_palavras.csv", index=False)
print("Tabela de comparação salva com sucesso!")

# Exibir as 50 primeiras palavras mais frequentes após a normalização
print(df_table_normalized.head(50))

# Criar um gráfico de barras para as 50 palavras mais frequentes
plt.figure(figsize=(20, 10))
ax = sns.barplot(x="Contagem", y="Palavra", data=df_table_normalized.head(50), palette="viridis")
plt.title("Top 50 Palavras Mais Frequentes Após Normalização", fontsize=15)
plt.xlabel("Ocorrência", fontsize=15)
plt.ylabel("Palavra", fontsize=15)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()

# Adicionar os valores ao lado de cada barra
for i, value in enumerate(df_table_normalized.head(50)["Contagem"]):
    ax.text(value + 1, i, str(value), color="black", va="center", fontsize=10)

# Salvar o gráfico
plt.savefig("grafico_barras_top_50_palavras_normalizadas.png")
plt.show()

# Salvar a tabela com todas as palavras normalizadas
df_table_normalized.to_csv("contagem_palavras_normalizadas.csv", index=False)
print("Tabela de palavras normalizadas salva com sucesso!")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import nltk
import re
import unicodedata

# Baixar os recursos necessários para lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

# Inicializar o lemmatizer
lemmatizer = WordNetLemmatizer()

# Função para normalizar palavras com transformação para o singular
def normalize_text(text):
    # Remover acentuação
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Transformar para minúsculas
    text = text.lower()
    # Remover caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Remover espaços extras
    text = text.strip()

    # Substituir palavras relacionadas a 'taxa' (evitar 'taxon')
    if re.search(r'\btax[a-z]*\b', text):  # Qualquer palavra que comece com 'tax'
        if text != "taxon":
            text = "taxa"

    # Aplicar lemmatization para converter para o singular
    text = lemmatizer.lemmatize(text)

    # Garantir que 'taxon' não seja mantido (convertido para 'taxa')
    if text == "taxon":
        text = "taxa"

    return text

# Carregar o dataset
df = pd.read_csv('/content/drive/My Drive/ASPECTS/Teste_prompt/processed_Cópia de subset_parte_1.csv')

# Dividir a coluna `document_topic` por linha ("/n") e criar colunas separadas
topics_split = df['document_topic'].str.split('\n', expand=True)

# Anexar as novas colunas ao DataFrame original
df = pd.concat([df, topics_split], axis=1)

# Identificar as colunas criadas a partir de `document_topic`
topic_columns = topics_split.columns

# Empilhar todas as colunas de tópicos em uma única série
topics_flat_before = df[topic_columns].stack()

# Remover valores nulos (antes da normalização)
topics_flat_before = topics_flat_before.dropna()

# Contar o número total de palavras únicas antes da normalização
word_counts_before = topics_flat_before.value_counts()
total_words_before = word_counts_before.shape[0]
print(f"Número total de palavras únicas antes da normalização: {total_words_before}")

# Aplicar a normalização em cada palavra com a nova função
topics_flat_after = topics_flat_before.apply(normalize_text)

# Substituir palavras mal grafadas ou variações manuais
topics_flat_after = topics_flat_after.replace({
    "passagen": "passagem",
    "passagens": "passagem",
    "voos": "voo",
    "bilhetes": "bilhete",
    # Adicione outros mapeamentos se necessário
})

# Contar as ocorrências de palavras únicas após a normalização
word_counts_after = topics_flat_after.value_counts()

# Exibir o número total de palavras únicas extraídas após normalização
total_words_after = word_counts_after.shape[0]
print(f"Número total de palavras únicas extraídas: {total_words_after}")

# Criar uma tabela de comparação antes e depois da normalização
comparison_table = pd.DataFrame({
    "Antes da Normalização": topics_flat_before,
    "Depois da Normalização": topics_flat_after
})

# Salvar a tabela de comparação
comparison_table.to_csv("comparacao_palavras_antes_depoisaaaaaaaaaaa.csv", index=False)

# Transformar a saída em um DataFrame para palavras após a normalização
df_table_after = word_counts_after.reset_index()
df_table_after.columns = ["Palavra", "Contagem"]

# Exibir os 20 primeiros registros para validação
print(df_table_after.head(100))

# Criar um gráfico de barras para as top 20 palavras
plt.figure(figsize=(20, 10))
ax = sns.barplot(x="Contagem", y="Palavra", data=df_table_after.head(100), palette="viridis")
plt.title("Top 20 Aspectos mais frequentes (versão com ajustes e singularização)", fontsize=15)
plt.xlabel("Ocorrência", fontsize=15)
plt.ylabel("Aspecto", fontsize=15)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()

# Adicionar os valores ao lado de cada barra
for i, value in enumerate(df_table_after.head(100)["Contagem"]):
    ax.text(value + 1, i, str(value), color="black", va="center", fontsize=15)

# Salvar o gráfico
plt.savefig("grafico_barras_top_20_palavras_normalizadas_ajustes.png")
plt.show()

# Salvar a tabela com palavras normalizadas
df_table_after.to_csv("contagem_palavras_completa_norm_ajustada.csv", index=False)

print("Tabela de contagem de palavras salva com sucesso!")
print("Tabela de comparação antes e depois da normalização salva com sucesso!")
